In [1]:
import habana_frameworks.torch.core as htcore
import torch
import torch.nn as nn

/usr/local/lib/python3.10/dist-packages/torch/distributed/distributed_c10d.py:366: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, OPTForCausalLM, AutoModelForCausalLM, BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    # prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
    PeftConfig,
    PeftModel
)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id="meta-llama/Llama-3.2-3B-Instruct"

lora_r=8
lora_alpha=16
lora_dropout=0.05
lora_target_modules=["q_proj","v_proj"]

In [4]:
device = torch.device('hpu')

In [5]:
config = LoraConfig(
            r=lora_r,
            lora_alpha=lora_alpha,
            target_modules=lora_target_modules,
            lora_dropout=lora_dropout,
            bias="none",
            task_type="CAUSAL_LM",
        )



Load 8-bit error

In [6]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True,
                                         llm_int8_threshold=200.0)

llm_model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device, torch_dtype=torch.float16, quantization_config=quantization_config,)
# llm_model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device, torch_dtype=torch.float16)
llm_tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)

llm_model = prepare_model_for_kbit_training(llm_model)

llm_model = get_peft_model(llm_model, config)
llm_model.print_trainable_parameters()

RuntimeError: No GPU found. A GPU is needed for quantization.

In [7]:
# quantization_config = BitsAndBytesConfig(load_in_8bit=True,
#                                          llm_int8_threshold=200.0)

# llm_model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device, torch_dtype=torch.float16, quantization_config=quantization_config,)
llm_model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device, torch_dtype=torch.float16)
llm_tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)

llm_model = prepare_model_for_kbit_training(llm_model)

llm_model = get_peft_model(llm_model, config)
llm_model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]============================= HABANA PT BRIDGE CONFIGURATION =========================== 
 PT_HPU_LAZY_MODE = 1
 PT_RECIPE_CACHE_PATH = 
 PT_CACHE_FOLDER_DELETE = 0
 PT_HPU_RECIPE_CACHE_CONFIG = 
 PT_HPU_MAX_COMPOUND_OP_SIZE = 9223372036854775807
 PT_HPU_LAZY_ACC_PAR_MODE = 1
 PT_HPU_ENABLE_REFINE_DYNAMIC_SHAPES = 0
---------------------------: System Configuration :---------------------------
Num CPU Cores : 160
CPU RAM       : 2113407632 KB
------------------------------------------------------------------------------
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.11s/it]


trainable params: 2,293,760 || all params: 3,215,043,584 || trainable%: 0.0713


LLM save Error

In [8]:
llm_model.save_pretrained('testing/', save_embedding_layers=True)

/usr/local/lib/python3.10/dist-packages/safetensors/torch.py:17: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return tensor.storage().data_ptr()


RuntimeError: Attempted to access the data pointer on an invalid python storage.

In [9]:
llm_model.to('cpu')
llm_model.save_pretrained('testing/', save_embedding_layers=True)
llm_model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in